In [1]:
from rnn.model import *
from rnn.train import *
from rnn.predict import *
from utils.data_loader import *

In [2]:
# Hyper params
hidden_size = 128
batch_size = 10
sequence_len = 100
lr = 0.01
n_layers = 1
n_epochs = -1
cycle = 100

In [3]:
# load data from file
#data loader
loader = data_loader(sequence_len, batch_size)
loader.load()

In [7]:
# load functionality check
cb = loader.cb
data_batch = loader.np_batchs
print("Input Vector Size : " + str(cb.size()))
print("Total Batch Num : " + str(len(data_batch)))
print(data_batch)

Input Vector Size : 272
Total Batch Num : 21023
[[[ 38   5   1 ...,  13 100   0]
  [ 11   3   1 ...,   0  18   9]
  [  3  16   6 ...,  13  19   7]
  ..., 
  [ 17   4   5 ...,  13  11  11]
  [  1   5  10 ...,  10   3   5]
  [ 16  12   1 ...,   8   4   5]]

 [[  2   3  14 ...,   9  14   0]
  [ 14  19   0 ...,  12  18   3]
  [  9   0  10 ...,   8   4  11]
  ..., 
  [  0  14  19 ...,   7   2   8]
  [  0   8  13 ...,   2   8   1]
  [  9   1  20 ...,   3   2   0]]

 [[  1  25   1 ...,   5   0  17]
  [  4   9   9 ...,  12  27   8]
  [  1   0  11 ...,   3  13   6]
  ..., 
  [  1   1   0 ...,   9   4  16]
  [  3   5  18 ...,  22  10   4]
  [ 17  24  20 ...,   6   3   5]]

 ..., 
 [[ 10  19   0 ...,   1   6   1]
  [  0   2   8 ...,  16   1   5]
  [  1   9   4 ...,   8   4   2]
  ..., 
  [  8   1   0 ...,   1   6  12]
  [ 10   7  24 ...,   2   3   0]
  [ 14  19   0 ...,   6   0  15]]

 [[  7  10  10 ...,   4  19  12]
  [  4   6   0 ...,   0   3   5]
  [ 10  19   0 ...,   8   2  23]
  ..., 
  [  8

In [9]:
# Required Params
input_size = cb.size()
output_size = cb.size()
start_sequence = cb.get_number("A")
# Model
decoder = RNN(input_size, hidden_size, output_size, n_layers)
optimizer = torch.optim.Adam(decoder.parameters(), lr)
criterion = nn.CrossEntropyLoss()
start = time.time()
all_losses = []
loss_avg = 0

try:
    if (n_epochs == -1):
        epoch = 0
        while (True):
            epoch += 1
            loss = train(decoder, optimizer, criterion, sequence_len, batch_size , *random_training_set(data_batch, sequence_len, batch_size))
            loss_avg += loss
            
            print(epoch)
            if epoch % cycle == 0:
                print('[Time : %s, Epoach : (%d), Loss : %.4f]' % (time_since(start), epoch, loss))
                print(predict(decoder, start_vector), '\n')
                #save(decoder, "./model/rnn_model"+str(time.time()))

    else :
        print("hi")
        
except KeyboardInterrupt:
    print("Saving before quit...")
    save(decoder, "./model/rnn_model"+str(time.time()))

[28]
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
Saving before quit...
Saved as rnn_model1512560107.pt
